In [30]:
# parse the page function
from bs4 import BeautifulSoup
import requests
import re
import os
import nltk

metadata = {}


def WordsToNouns (lines):
    # note may need to run nltk import (see bottom cell in this noteboox)
    words = set(nltk.corpus.words.words())
    # function to test if something is a noun
    is_noun = lambda pos: pos[:2] == 'NN'
    # do the nlp stuff
    tokenized = nltk.word_tokenize(lines)
    # the line below checks if in the word dictionary
    #nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos) and word.lower() in words]
    nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]
    # remove duplicates
    names = set(nouns)
    return names


def ParsePage(file):
    r  = BeautifulSoup(open(file), "html.parser")
    data = r.text
    data = data.replace('\n', '#')
    data=re.sub(r'#+','#',data)

    keywords = ['Value','Issued by','Date','Printer']
    fields = data
    meta = data.split('#')

    print(meta)

    # all the countries on this website are uppercase
    # so if the markup is wrong just ignore it
    if meta[3].islower() or meta[3].startswith(')') :
        return None
    
   #printer_pos = meta.index('Printer') + 1

    metadata['pick']=meta[1];
    
    # if there is no pick number - forget it!
    if not metadata['pick'].startswith('P'):
        return None

    value_pos = meta.index('Value') + 1
    issuedby_pos = meta.index('Issued by') + 1
    date_pos = meta.index('Date') + 1
    desc_pos_start = date_pos
    #get the index if the start of the item in the list has BACK in it
    desc_pos_end = [i for i, item in enumerate(meta) if re.search('BACK',item)][0]
    desc = meta[desc_pos_start:desc_pos_end]
    desc = ','.join(desc)
                    
    print("desc_pos_start ",desc_pos_start,"desc_pos_end",desc_pos_end)
    print(WordsToNouns(desc))
        
    
    metadata['country']=meta[3] or None;
    metadata['value']=meta[value_pos];
    
    value = meta[value_pos];
    
    if re.search(r"(\d+)", value):    
        metadata['value']=re.search(r"(\d+)", value).group(1)
    else:
        metadata['value']=None
        
    if re.search(r"([^\d\W]+)",value):    
        metadata['currency']=re.search(r"([^\d\W]+)", value).group(1)
    else:
        metadata['currency']=None
                 
    metadata['issued by']=meta[issuedby_pos] or None;
    date=meta[date_pos];
    
    try:
        metadata['start date']=re.search(r"(\d{4})", date).group(1)
    except AttributeError:
        metadata['start date']=None
        
    #metadata['printer']=meta[printer_pos];

    images = r.findAll('img')
    image_names = []
    for image in images:
        #print image source
        image_names.append(image['src'])
        
    try:
        metadata['obverse'] = image_names[0]
        metadata['reverse'] = image_names[1]
        metadata['imgpath'] = os.path.dirname(file)
        metadata['fullpath'] = file
        return (metadata)
    except IndexError:
        return None
    
file = "C:\\Users\\staylor\\Documents\\MobaXterm\\home\\banknotes\\banknote.ws\\COLLECTION\\countries\\AFR\\ALG\\ALG0097.htm"
print(ParsePage(file))


['', 'P-97', '97', 'ALGERIA', 'Value', '50 Centimes = 10 Surdi (Sou, Soldi)', 'Issued by', 'Trésorerie - Région Économique ', "    d'Algérie", 'Date', 'L. 1944', '    for a similar note of the 2nd issue (2e Tranche) see ', '    ALGERIA P-100 ', 'Printer', 'La Typo-Litho J. Carbonel, Algiers', '\xa0\xa0', '\xa0', '97a', 'series C, C1-C4', '97b', 'series F, F1', 'BACK TO ALGERIA', 'BACK TO AFRICA', 'BACK TO INDEX', '']
desc_pos_start  10 desc_pos_end 21
{'F', 'C1-C4,97b', 'note', 'L.', 'J.', 'Tranche', 'F1', 'Typo-Litho', 'ALGERIA', 'La', 'Algiers', ',97a', 'P-100', 'series', 'Printer', 'C', 'issue', 'Carbonel'}
{'pick': 'P-97', 'country': 'ALGERIA', 'value': '50', 'currency': 'Centimes', 'issued by': 'Trésorerie - Région Économique ', 'start date': '1944', 'obverse': 'ALG0097ao.JPG', 'reverse': 'ALG0097ar.JPG', 'imgpath': 'C:\\Users\\staylor\\Documents\\MobaXterm\\home\\banknotes\\banknote.ws\\COLLECTION\\countries\\AFR\\ALG', 'fullpath': 'C:\\Users\\staylor\\Documents\\MobaXterm\\home\

In [4]:
# get all the html pages
import os
import glob


def GetHTMLPages (root_dir):
    pages = []
    for root, dirs, files in os.walk(root_dir):
        path = root.split(os.sep)
        #print((len(path) - 1) * '---', os.path.basename(root))
        for file in files:
            if re.search(r'\d+\.htm$', file):
                full_path = root + "\\" + file
                #print(len(path) * '---', file)
                pages.append(full_path)
    return pages


pages = []
root_dir = 'c:\\Users\\staylor\\Documents\\MobaXterm\\home\\banknotes'
pages = GetHTMLPages(root_dir)
test_pages = (pages[0:10])
print(test_pages)
all_pages = pages

['c:\\Users\\staylor\\Documents\\MobaXterm\\home\\banknotes\\banknote.ws\\COLLECTION\\countries\\AFR\\ALG\\ALG0003.htm', 'c:\\Users\\staylor\\Documents\\MobaXterm\\home\\banknotes\\banknote.ws\\COLLECTION\\countries\\AFR\\ALG\\ALG0005.htm', 'c:\\Users\\staylor\\Documents\\MobaXterm\\home\\banknotes\\banknote.ws\\COLLECTION\\countries\\AFR\\ALG\\ALG0006.htm', 'c:\\Users\\staylor\\Documents\\MobaXterm\\home\\banknotes\\banknote.ws\\COLLECTION\\countries\\AFR\\ALG\\ALG0010.htm', 'c:\\Users\\staylor\\Documents\\MobaXterm\\home\\banknotes\\banknote.ws\\COLLECTION\\countries\\AFR\\ALG\\ALG0011.htm', 'c:\\Users\\staylor\\Documents\\MobaXterm\\home\\banknotes\\banknote.ws\\COLLECTION\\countries\\AFR\\ALG\\ALG0013.htm', 'c:\\Users\\staylor\\Documents\\MobaXterm\\home\\banknotes\\banknote.ws\\COLLECTION\\countries\\AFR\\ALG\\ALG0014.htm', 'c:\\Users\\staylor\\Documents\\MobaXterm\\home\\banknotes\\banknote.ws\\COLLECTION\\countries\\AFR\\ALG\\ALG0015.htm', 'c:\\Users\\staylor\\Documents\\MobaXte

In [5]:
# for testing
for page in test_pages:
    #print(page)
    res = ParsePage(page)
    if res != None:
        for key,value in ParsePage(page).items() :
            print(value,"\t", end='', flush=True)
            
    print("\n")
    

['', 'P-3', '3', 'ALGERIA', 'Value', '100 Francs', 'Issued by', "Banque de l'Algérie", 'Date', '1852-1860', 'Remark', 'proofs only do exist', '\xa0', '\xa0', '\xa0Caduceus, cornucopiæ, ', "  allegorical women, man with ship's rudder and mast, Neptune;", '"France" greeting Arab between two reclining women', '\xa0', 'BACK TO ALGERIA', 'BACK TO AFRICA', 'BACK TO INDEX', '']


['', 'P-5', '5', 'ALGERIA', 'Value', '500 Francs', 'Issued by', "Banque de l'Algérie", 'Date', '1852-1860', 'Remark', 'proofs only do exist', '\xa0', '\xa0', 'Caduceus, cornucopiæ, ', "  allegorical women, man with ship's rudder and mast, Neptune", 'Caduceus, seal', 'BACK TO ALGERIA', 'BACK TO AFRICA', 'BACK TO INDEX', '']


['', 'P-6', '6', 'ALGERIA', 'Value', '1.000 Francs', 'Issued by', "Banque de l'Algérie", 'Date', '16.10.1870', 'Remark', 'remainder', 'Caduceus, cornucopiæ, ', "  allegorical women, man with ship's rudder and mast, Neptune", '\xa0', 'BACK TO ALGERIA', 'BACK TO AFRICA', 'BACK TO INDEX', '']
['', '

In [5]:
# parse all the htm files and write out tsv
import pandas as pd
list_dict = []

for page in all_pages:
    print(page)
    try:
        res = ParsePage(page)
    except:
        continue
    if res != None:
        #print(">>>>>")
        #print(res)
        #for key,value in res.items() :
        #    print(value,"\t", end='', flush=True)
        # need to make a copy of the dict to add to the list
        list_dict.append(res.copy()) 
    #print("\n")
    
    
df = pd.DataFrame(list_dict)
df.head()

zegami_tsv = "zegami_banknotes.tsv"
df.to_csv(zegami_tsv, sep='\t', index=False)

NameError: name 'all_pages' is not defined

In [31]:
# copy all files in the df to a new directory
# some won't copy due to errors in the data
import shutil
import os


destination = "c:\\Users\\staylor\\Documents\\banknotesimages\\"

df2=df[:10]
#print(df2)

for index, row in df.iterrows():
    #print(df['imgpath']+df['obverse'])
    #source_file = df2['imgpath']+"\\"+df2['obverse']
    path=row['imgpath']
    file=row['obverse']
    src=path+"\\"+file
    try:
        shutil.copy(src,destination)
    except OSError:
        continue
    

In [28]:
import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
nltk.download('words')

def WordsToNouns (lines):
    # function to test if something is a noun
    is_noun = lambda pos: pos[:2] == 'NN'
    # do the nlp stuff
    tokenized = nltk.word_tokenize(lines)
    nouns = [word for (word, pos) in nltk.pos_tag(tokenized) if is_noun(pos)]
    # remove duplicates
    names = set(nouns)
    return names

lines = '  allegorical women, man with ships rudder and mast, Neptune, Windrose, seal, 11r, not issued remainder, 11a, issued note without overprint, 11b, vertical overprint at left: Succursale de ,     Oran, 11c, vertical overprint at left: Succursale de ,     Bone, 11d, vertical overprint at left: Succursale de ,     Constantine,     for a cancelled note from Constantine see ALGERIA P-38,'
print(WordsToNouns(lines))



[nltk_data] Downloading package words to
[nltk_data]     C:\Users\staylor\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.
{'Bone', 'note', 'remainder', 'Windrose', 'rudder', 'Oran', 'ALGERIA', 'women', 'left', 'seal', 'man', 'Neptune', 'Constantine', 'mast', 'ships', 'Succursale', 'P-38', 'overprint'}
